In [2]:
import requests
import os
import pandas as pd
from PIL import Image
# Load the test set
test_df = pd.read_csv('test.csv')
calibration = "00"
pdf_path = "./test.pdf"
# Define the base URL of your API
base_url = 'http://127.0.0.1:8080'


def image_to_pdf(image_path, output_pdf_path):
    # Open the image file
    image = Image.open(image_path)
    
    # Convert image to RGB (if not already in RGB mode)
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    # Save image as PDF
    image.save(output_pdf_path, "PDF", resolution=100.0)

def send_pdf_to_api(pdf_path, api_url, headers=None):
    # Open the PDF file in binary mode
    with open(pdf_path, 'rb') as pdf_file:
        # Prepare the file data to send in the request
        files = {'file': (os.path.basename(pdf_path), pdf_file, 'application/pdf')}
        
        # Send the POST request to the API
        response = requests.post(f"{base_url}/{api_url}", files=files, headers=headers)
        
    return response

In [3]:
test_df = test_df[test_df['label'] != "diskutera"]

In [ ]:
test_df["label"].unique()

In [5]:
def check_skalstock(image_path, data, endpoint):
    # Convert the image to a PDF
    #print(data)
    image_to_pdf(image_path, pdf_path)
    
    # Send the PDF to the API
    response = send_pdf_to_api(pdf_path, endpoint)

    response_dict = response.json()
    print(response_dict)
    if response_dict['scale']["status"]:
        scale_api = 1
    else:
        scale_api = 0
    if "skalstock_ok" == data["label"] or "skalstock_ej_ok" == data["label"]:
        print("scale")
        scale_true = 1
    else:
        scale_true = 0

    return scale_true, scale_api

In [ ]:
data = test_df[test_df["label"].isin(["skalstock_ok", "skalstock_saknas", "skalstock_ej_ok"])]
results = {"Fasad": {"scale": {"T":[], "P":[], "image":[]}}}
data
for i in range(len(data)):
    print(i, "/", len(data))
    row = data.iloc[i]
    image = row["image"]
    if "Fasadritning" in image:
        scale_true, scale_api = check_skalstock(image, row, "checkFasadritning")
        
    elif "Planritning" in image:
        scale_true, scale_api = check_skalstock(image, row, "checkPlanritning")
    elif "Sektion" in image:
        scale_true, scale_api = check_skalstock(image, row, "checkSektionsritning")
    elif "Situation" in image:
        scale_true, scale_api = check_skalstock(image, row, "checkSituationsplan")
    elif "Nybyggnadskarta" in image:
        scale_true, scale_api = check_skalstock(image, row, "checkSituationsplan")
    else:
        print("No match", image)
        continue
    #print(results)
    print(scale_true, scale_api)
    results["Fasad"]["scale"]["T"].append(scale_true)
    results["Fasad"]["scale"]["P"].append(scale_api)
    results["Fasad"]["scale"]["image"].append(image)
    print(scale_true, scale_api, image)

In [156]:
result_df = pd.DataFrame(results["Fasad"]["scale"])

In [157]:
result_df = pd.DataFrame(results["Fasad"]["scale"])
result_df.to_csv(f"skalstock_results{calibration}.csv", index=False)

In [ ]:
#draw confusion matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
confusion_matrix(result_df["T"], result_df["P"])
cm = confusion_matrix(result_df["T"], result_df["P"])
sns.heatmap(cm, annot=True, fmt='g')
plt.title('Confusion matrix för skalstock på alla ritningar')
plt.xlabel("AI")
plt.ylabel("Anoterad")
plt.show()


In [ ]:
#for each image type draw a confusion matrix

for image_type in ["Fasad", "Plan", "Sektion", "Situation", "Nybyggnadskarta"]:
    resultat_type = result_df[result_df["image"].str.contains(image_type)]
    cm = confusion_matrix(resultat_type["T"], resultat_type["P"])
    sns.heatmap(cm, annot=True, fmt='g')
    plt.title(f'Confusion matrix för skalstock på {image_type}')
    plt.xlabel("AI")
    plt.ylabel("Anoterad")
    plt.show()
    

In [160]:
def check_kompass(image_path, data, endpoint):
    # Convert the image to a PDF
    #print(data)
    image_to_pdf(image_path, pdf_path)
    
    # Send the PDF to the API
    response = send_pdf_to_api(pdf_path, endpoint)

    response_dict = response.json()
    print(response_dict)
    if response_dict['direction_marker']["status"]:
        direction_api = 1
    else:
        direction_api = 0
    if "riktning_symbol" == data["label"]:
        print("scale")
        direction_true = 1
    else:
        direction_true = 0

    return direction_true, direction_api

In [ ]:
test_df["label"].unique()

In [ ]:
data = test_df[test_df["label"].isin(["riktning_symbol", "riktning_saknas", "riktning_ej_ok"])]
results = {"Direction": {"T":[], "P":[], "image":[]}}

for i in range(len(data)):
    print(i, "/", len(data))
    row = data.iloc[i]
    image = row["image"]
    if "Situation" in image:
        direction_true, direction_api = check_kompass(image, row, "checkSituationsplan")
    elif "Nybyggnadskarta" in image:
        direction_true, direction_api = check_kompass(image, row, "checkSituationsplan")
    else:
        print("No match", image)
        continue
    #print(results)
    print(row["label"])
    results["Direction"]["T"].append(direction_true)
    results["Direction"]["P"].append(direction_api)
    results["Direction"]["image"].append(image)
    print(direction_true, direction_api, image)

In [163]:
result_df = pd.DataFrame(results["Direction"])
result_df.to_csv(f"kompassros_results{calibration}.csv", index=False)

In [ ]:
#draw confusion matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
confusion_matrix(result_df["T"], result_df["P"])
cm = confusion_matrix(result_df["T"], result_df["P"])
sns.heatmap(cm, annot=True, fmt='g')
plt.title('Confusion matrix för kompassros på alla ritningar')
plt.xlabel("AI")
plt.ylabel("Anoterad")
plt.show()


In [165]:
def check_riktning(image_path, data, endpoint):
    # Convert the image to a PDF
    #print(data)
    image_to_pdf(image_path, pdf_path)
    
    # Send the PDF to the API
    response = send_pdf_to_api(pdf_path, endpoint)

    response_dict = response.json()
    print(response_dict)
    if response_dict['direction_marker']["status"]:
        direction_api = 1
    elif response_dict['direction_marker']["code"] == 240:
        direction_api = 2
    elif response_dict['direction_marker']["code"] == 250:
        direction_api = -1
    else:
        direction_api = 0
    num_fasad = len(data[data["label"] == "fasad"])
    num_riktning = len(data[data["label"] == "riktning_text"])
    if num_fasad == num_riktning:
        direction_true = 1
    else:
        direction_true = 0
    print(num_fasad, num_riktning)
    return direction_true, direction_api

In [ ]:
test_df["label"].unique()

In [ ]:
data = test_df[test_df["image"].str.contains("Fasadritning")]


results = {"Direction": {"T":[], "P":[], "image":[]}}
images = list(data["image"].unique())

data.groupby("image")


for i in range(len(images)):
    print(i, "/", len(images))
    image = images[i]
    rows = data[data["image"] == image]
    if ("riktning_text" in rows["label"].unique() or "riktning_saknas" in rows["label"].unique() or "riktning_ej_ok" in rows["label"].unique()) and "fasad" in rows["label"].unique():
        if "Fasadritning" in image:
            direction_true, direction_api = check_riktning(image, rows, "checkFasadritning")
        else:
            print("No match", image)
            continue
    else:
        print("No match", image)
        continue
    #print(results)
    #print(row["label"])
    results["Direction"]["T"].append(direction_true)
    results["Direction"]["P"].append(direction_api)
    results["Direction"]["image"].append(image)
    print(direction_true, direction_api, image)

In [168]:
result_df = pd.DataFrame(results["Direction"])
result_df.to_csv(f"riktning_text_results{calibration}.csv", index=False)

In [ ]:
#draw confusion matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
confusion_matrix(result_df["T"], result_df["P"])
cm = confusion_matrix(result_df["T"], result_df["P"])
sns.heatmap(cm, annot=True, fmt='g')
plt.title('Confusion matrix för kompassriktning på alla ritningar')
plt.xlabel("AI")
plt.ylabel("Anoterad")
plt.show()

In [170]:
def check_distance(image_path, data, endpoint):
    # Convert the image to a PDF
    #print(data)
    image_to_pdf(image_path, pdf_path)
    
    # Send the PDF to the API
    response = send_pdf_to_api(pdf_path, endpoint)

    response_dict = response.json()
    print(response_dict)
    if response_dict['distance_messurement']["code"] == "400":
        distance_api = 1
        num_lines_api = 3
    elif response_dict['distance_messurement']["code"] == "440":
        distance_api = 0
        num_lines_api = 0
    elif response_dict['distance_messurement']["code"] == "460":
        distance_api = 0
        num_lines_api = 1
    elif response_dict['distance_messurement']["code"] == "470":
        distance_api = 0
        num_lines_api = 2
    else:
        distance_api = 0
    lines = ['linje1', 'linje2','linje3', 'linje4', 'linje5', 'linje6']
    num_lines = 0
    for line in lines:
        line_annotation = data[data["label"] == line]
        if len(line_annotation) == 2:
            num_lines += 1
    if num_lines >= 3:
        distance_true = 1
    else:
        distance_true = 0
    print("num lines ", num_lines)
    if num_lines > 3:
        num_lines = 3
    return distance_true, distance_api, num_lines_api, num_lines

In [ ]:
test_df[test_df["label"] == "linje1"]

In [ ]:
data = test_df[test_df["label"] == "linje1"]


results = {"Distance": {"T":[], "P":[],"Num_lines_api": [], "Num_lines_annotation":[], "image":[]}}
images = list(data["image"].unique())


for i in range(len(images)):
    print(i, "/", len(images))
    image = images[i]
    rows = test_df[test_df["image"] == image]
    print(rows["label"].unique())
    print(len(rows))
    if "linje1" in rows["label"].unique():
        if "Situationsplan" in image or "Nybyggnadskarta" in image:
            direction_true, direction_api, num_lines_api, num_lines_true = check_distance(image, rows, "checkSituationsplan")
        else:
            print("No match", image)
            continue
    else:
        print("No match", image)
        continue
    #print(results)
    #print(row["label"])
    results["Distance"]["T"].append(direction_true)
    results["Distance"]["P"].append(direction_api)
    results["Distance"]["Num_lines_api"].append(num_lines_api)
    results["Distance"]["Num_lines_annotation"].append(num_lines_true)
    results["Distance"]["image"].append(image)
    print(direction_true, direction_api, image, num_lines_api, num_lines_true)
    print("1:" ,results["Distance"]["T"].count(1), results["Distance"]["P"].count(1))
    print("0:", results["Distance"]["T"].count(0), results["Distance"]["P"].count(0))

In [173]:
result_df = pd.DataFrame(results["Distance"])
result_df.to_csv(f"distance_results{calibration}.csv", index=False)

In [ ]:
result_df

In [ ]:
#draw confusion matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
confusion_matrix(result_df["Num_lines_annotation"], result_df["Num_lines_api"])
cm = confusion_matrix(result_df["Num_lines_annotation"], result_df["Num_lines_api"])
sns.heatmap(cm, annot=True, fmt='g')
plt.title('Confusion matrix för Avstånd till tomt på alla ritningar')
plt.xlabel("AI")
plt.ylabel("Anoterad")
plt.show()

In [ ]:
#draw confusion matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
confusion_matrix(result_df["T"], result_df["P"])
cm = confusion_matrix(result_df["T"], result_df["P"])
sns.heatmap(cm, annot=True, fmt='g')
plt.title('Confusion matrix för kompassriktning på alla ritningar')
plt.xlabel("AI")
plt.ylabel("Anoterad")
plt.show()

In [26]:
def check_groundlines(image_path, data, endpoint, ground_line=True):
    # Convert the image to a PDF
    #print(data)
    image_to_pdf(image_path, pdf_path)
    
    # Send the PDF to the API
    response = send_pdf_to_api(pdf_path, endpoint)

    response_dict = response.json()
    print(response_dict)
    if response_dict['ground_level']["code"] == "300":
        ground_api = 1
        num_lines_api = 1
    elif response_dict['ground_level']["code"] == "340":
        ground_api = 1
        num_lines_api = 2

    else:
        num_lines_api = 0
        ground_api = 0

  
    num_fasad = len(data[data["label"] == "fasad"])
    num_ground = len(data[data["label"] == "ny_marklinje"])
    num_lines = 0
    if ground_line:
        for i in range(num_fasad):
            # check if ther is a marklinje within the borders of the fasad
            for j in range(num_ground):
                x1 = data[data["label"] == "fasad"].iloc[i]["x"]
                x2 = data[data["label"] == "fasad"].iloc[i]["x"] + data[data["label"] == "fasad"].iloc[i]["width"]
                y1 = data[data["label"] == "fasad"].iloc[i]["y"]
                y2 = data[data["label"] == "fasad"].iloc[i]["y"] + data[data["label"] == "fasad"].iloc[i]["height"]
                x = data[data["label"] == "ny_marklinje"].iloc[j]["x"]
                y = data[data["label"] == "ny_marklinje"].iloc[j]["y"]
                if x > x1 and x < x2 and y > y1 and y < y2:
                    num_lines += 1
                    break
    else:
        ground_lines = 0
            
    if num_lines >= num_fasad:
        ground_true = 1
    else:
        ground_true = 0
    return ground_true, ground_api, num_lines_api, num_lines

In [ ]:
test_df.label.unique()

In [ ]:
data = test_df[test_df["label"] == "fasad"]


results = {"Distance": {"T":[], "P":[],"Num_lines_api": [], "Num_lines_annotation":[], "image":[]}}
images = list(data["image"].unique())


for i in range(len(images)):
    print(i, "/", len(images))
    image = images[i]
    rows = test_df[test_df["image"] == image]
    print(rows["label"].unique())
    print(len(rows))
    if "ny_marklinje" in rows["label"].unique():
        if "Fasadritning" in image :
            direction_true, direction_api, num_lines_api, num_lines_true = check_groundlines(image, rows, "checkFasadritning")
            print(direction_true, direction_api, num_lines_api, num_lines_true)
        else:
            print("No match", image)
            continue
    else:
        direction_true, direction_api, num_lines_api, num_lines_true = check_groundlines(image, rows, "checkFasadritning", ground_line = False)
        
    #print(results)
    #print(row["label"])
    results["Distance"]["T"].append(direction_true)
    results["Distance"]["P"].append(direction_api)
    results["Distance"]["Num_lines_api"].append(num_lines_api)
    results["Distance"]["Num_lines_annotation"].append(num_lines_true)
    results["Distance"]["image"].append(image)
    print(direction_true, direction_api, image, num_lines_api, num_lines_true)
    print("1:" ,results["Distance"]["T"].count(1), results["Distance"]["P"].count(1))
    print("0:", results["Distance"]["T"].count(0), results["Distance"]["P"].count(0))

In [ ]:
calibration = "06"

In [29]:
result_df = pd.DataFrame(results["Distance"])
result_df.to_csv(f"ground_result{calibration}.csv", index=False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
confusion_matrix(result_df["T"], result_df["P"])
cm = confusion_matrix(result_df["T"], result_df["P"])
sns.heatmap(cm, annot=True, fmt='g')
plt.title('Confusion matrix för marklinjer på alla ritningar')
plt.xlabel("AI")
plt.ylabel("Annoterad")
plt.show()